# **Värmeväxlarens rörlängdsberäknare**

Detta Jupyter Notebook är skapad av ChatGPT, jag har bara kikat igenom koden så den ser halvvettig ut. 

---
### **Nyckelfunktioner**
Du kan lätt beräkna den nödvändiga rörlängden för en markkopplad värmeväxlare genom slides som kan justera slidesen, vill du inte och pilla kan du in gå in i koden också.
- **Dynamiska inmatningar**: Justera parametrarna interaktivt för att se hur de påverkar den nödvändiga rörlängden.
- **Hanterar både uppvärmning och kylning**: Säkerställer korrekta beräkningar för både positiva och negativa temperaturskillnader.
- **Tryckfall**: Beräknar tryckfallet över röret med Darcy-Weisbachs ekvation.

---
### **Hur man använder det**
1. Justera reglagen för inmatningsvärden.
2. Kör notebook-cellen för att se uppdaterade resultat.
3. Granska nyckelutdata, inklusive nödvändig rörlängd, massflöde, tryckfall och värmeöverföringsegenskaper.

---
### **Antaganden i koden**
1. **Luftegenskaper**:
   - Luftens densitet (ρ) antas vara 1,2 kg/m³.
   - Specifik värmekapacitet (ρ) för luft är 1005 J/(kg·K).
   - Luftens värmeledning (κ) är 0,025 W/(m·K).
   - Luftens dynamiska viskositet (µ) är 1,81e-5 kg/(m·s).

2. **Mark- och rörmaterial**:
   - Rörets tjocklek antas vara 3 mm (0,003 m).
   - Värmeledningsförmåga för rörmaterialet (κ) antas vara 0,4 W/(m·K).
   - Markens värmeledningsförmåga (κ) antas vara 1,5 W/(m·K).

3. **Strömningsantaganden**:
   - Flödestyp (laminärt eller turbulent) bestäms baserat på Reynolds tal.
   - Vid turbulent flöde används Blasius ekvation för friktionsfaktorn och Dittus-Boelter equation för nusselt.
   - Vid laminärt flöde antas friktionsfaktorn vara 64/Re.

---


In [ ]:
import numpy as np
from ipywidgets import interact, FloatSlider, IntSlider

# Function to calculate pressure drop

def calculate_pipe_length_dynamic(T_in, T_ground, v, D, delta_T_target):
    # Constants
    rho_air = 1.2  # Air density (kg/m³)
    cp_air = 1005  # Specific heat capacity of air (J/(kg·K))
    k_air = 0.025  # Thermal conductivity of air (W/(m·K))
    mu_air = 1.81e-5  # Dynamic viscosity of air (kg/(m·s))
    Pr_air = 0.7  # Prandtl number (dimensionless)

    # Target outlet temperature based on the target temperature difference
    if T_in > T_ground:  # Cooling case
        T_out = T_ground + delta_T_target
    else:  # Heating case
        T_out = T_ground - delta_T_target

    # Effectiveness
    effectiveness = (T_out - T_in) / (T_ground - T_in)

    # Cross-sectional area and mass flow rate
    A_cross = np.pi * (D / 2)**2  # Cross-sectional area (m²)
    m_dot = rho_air * v * A_cross  # Mass flow rate (kg/s)
    volumetric_flow_rate_m3h = m_dot / rho_air * 3600  # Volumetric flow rate (m³/h)

    # Reynolds number
    Re = (rho_air * v * D) / mu_air
    if Re > 2300:
        flow_type = "Turbulent"
        Nu = 0.023 * (Re**0.8) * (Pr_air**0.3)  # Turbulent Nusselt number, Dittus-Boelter equation
    else:
        flow_type = "Laminar"
        Nu = 3.66  # Laminar Nusselt number

    # Convective heat transfer coefficient
    h_conv = (Nu * k_air) / D

    # Overall heat transfer coefficient (soil properties assumed)
    t_tube = 0.003  # Tube wall thickness (m)
    k_tube = 0.4  # Thermal conductivity of tube material (W/(m·K))
    k_soil = 1.5  # Soil thermal conductivity (W/(m·K))
    R_cond = t_tube / (k_tube * np.pi * D)
    R_soil = 1 / (2 * np.pi * k_soil)
    U = 1 / (1 / h_conv + R_cond + R_soil)

    # Pipe length calculation
    L = - (m_dot * cp_air) / (U * np.pi * D) * np.log(1 - effectiveness)

    # Pressure drop calculation (Darcy-Weisbach equation)
    if Re > 2300:
        f = 0.079 * Re**-0.25  # Blasius correlation for turbulent flow
    else:
        f = 64 / Re  # Laminar flow friction factor

    pressure_drop = f * (L / D) * 0.5 * rho_air * v**2  # Pressure drop in Pascals

    # Results
    return {
        "Flow Type": flow_type,
        "Reynolds Number": Re,
        "Convective Heat Transfer Coefficient (h_conv)": h_conv,
        "Overall Heat Transfer Coefficient (U)": U,
        "Mass Flow Rate (m_dot, kg/s)": m_dot,
        "Volumetric Flow Rate (m³/h)": volumetric_flow_rate_m3h,
        "Target Outlet Temperature (T_out)": T_out,
        "Effectiveness": effectiveness,
        "Required Pipe Length (L)": L,
        "Pressure Drop (Pa)": pressure_drop
    }

# Function to display results
def display_results(T_in, T_ground, v, D, delta_T_target):
    results = calculate_pipe_length_dynamic(T_in, T_ground, v, D, delta_T_target)
    print("Värmeväxlarens beräkningsresultat")
    print("-" * 40)
    for key, value in results.items():
        if isinstance(value, float):
            print(f"{key}: {value:.2f}")
        else:
            print(f"{key}: {value}")

# Interactive sliders for input parameters
interact(
    display_results,
    T_in=FloatSlider(value=20, min=-30, max=50, step=1, description="T_in (°C)"),
    T_ground=FloatSlider(value=10, min=-10, max=40, step=1, description="T_mark (°C)"),
    v=FloatSlider(value=1, min=0.1, max=5, step=0.1, description="Hastighet (m/s)"),
    D=FloatSlider(value=0.11, min=0.05, max=0.5, step=0.01, description="Diameter (m)"),
    delta_T_target=FloatSlider(value=2, min=0.1, max=10, step=0.1, description="Delta T (°C)")
);

